
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>

<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.5 - alpha</p>
<p>Package compatibility: squid-py v0.5.4, keeper-contracts 0.8.6, utilities 0.2.0,
<p>Component compatibility: Brizo v0.2.9, Aquarius v0.1.11, Nile testnet smart contracts 0.8.6</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

# Getting Underway - Publishing assets
In this notebook, we will explore how to publish an Asset using Ocean Protocol. An Asset consists of several files
which are kept private, and optionally other links which are open (samples, descriptions, etc.).

A publisher will require access to two services;
1. A service to store the MetaData of the asset (part of the DDO) - 'Aquarius'
1. A service to manage permissioned access to the assets - 'Brizo'

The publishing of an asset consists of;
1. Preparing the asset files locally
1. Preparing the metadata of the asset
1. Uploading assets or otherwise making them available as URL's
1. Registering the metadata and service endpoints into Aquarius
1. Registering the asset into the Blockchain (into the DID Registry)

<p><img src="https://raw.githubusercontent.com/oceanprotocol/mantaray/develop/doc/img/jupyter_cell.png" alt="drawing" width="400" align="center"/></p>
<p><b>Overall client and service architecture</b></p>

### Section 0: Import modules, connect the Ocean Protocol API

In [1]:
# Standard imports
import logging

# Import mantaray and the Ocean API (squid)
import random
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
from mantaray_utilities.user import password_map
from pprint import pprint
# Setup logging
manta_utils.logging.logger.setLevel('INFO')
from time import sleep
print("squid-py Ocean API version:", squid_py.__version__)

10 - logging         - <module>        - Logging started


Using default logging settings.
squid-py Ocean API version: 0.5.11


In [2]:
# Get the configuration file path for this environment
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))

50 - <ipython-input-2-84136f362abd> - <module>        - Deployment type: USE_K8S_CLUSTER
50 - <ipython-input-2-84136f362abd> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini
50 - <ipython-input-2-84136f362abd> - <module>        - Squid API version: 0.5.11


In [3]:
# Instantiate Ocean with the default configuration file.
configuration = Config(CONFIG_INI_PATH)
ocn = Ocean(configuration)

20 - ocean_secret_store - __init__        - 	SecretStore: url https://secret-store.dev-ocean.com, parity-client https://nile.dev-ocean.com, account 0x44E07D8190Cc0f0eCCC9C935D5EA47194301Cb00
20 - diagnostics     - verify_contracts - Keeper contract artifacts (JSON abi files) at: /home/batman/ocn/mantaray_jupyter/artifacts_nile
20 - diagnostics     - verify_contracts - Using keeper contracts from network nile, network id is 8995
20 - diagnostics     - verify_contracts - Looking for keeper contracts ending with ".nile.json", e.g. "AgreementStoreManager.nile.json".
20 - diagnostics     - verify_contracts - Finished loading keeper contracts:
	Dispenser: 0x865396b7ddc58C693db7FCAD1168E3BD95Fe3368
	OceanToken: 0x9861Da395d7da984D5E8C712c2EDE44b41F777Ad
	DIDRegistry: 0x4A0f7F763B1A7937aED21D63b2A78adc89c5Db23
	AgreementStoreManager: 0x62f84700b1A0ea6Bfb505aDC3c0286B7944D247C
	TemplateStoreManager: 0x9768c8ae44f1dc81cAA98F48792aA5730cAd2F73
	ConditionStoreManager: 0x39b0AA775496C5ebf26f3B81C9e

### Section 1: A publisher account in Ocean

In [4]:
# Get a publisher account
path_passwords = manta_utils.config.get_project_path() / 'passwords.csv'
passwords = manta_utils.user.load_passwords(path_passwords)

publisher_acct = random.choice([acct for acct in ocn.accounts.list() if password_map(acct.address, passwords)])
publisher_acct.password = password_map(publisher_acct.address, passwords)
assert publisher_acct.password

20 - user            - load_passwords  - 31 account-password pairs loaded


In [5]:
print("Publisher account address: {}".format(publisher_acct.address))
print("Publisher account balance:", ocn.accounts.balance(publisher_acct).ocn)

Publisher account address: 0x442aCf84E05459e13595f372C49D4a15E95eA4E7
Publisher account balance: 0


Your account will need some Ocean Token to make real transactions, let's ensure that you are funded!

In [6]:
# ensure Ocean token balance
if ocn.accounts.balance(publisher_acct).ocn == 0:
    ocn.accounts.request_tokens(publisher_acct, 100)

### Section 2: Create the Metadata for your asset
The metadata is a key-value set of attributes which describe your asset

A more complex use case is to manually generate your metadata conforming to Ocean standard, but for demonstration purposes,
a utility in squid-py is used to generate a sample Meta Data dictionary.

In [7]:
# Get a simple example of Meta Data from the library directly
metadata = squid_py.ddo.metadata.Metadata.get_example()
print('Name of asset:', metadata['base']['name'])
# Print the entire (JSON) dictionary
pprint(metadata)

Name of asset: Ocean protocol white paper
{'additionalInformation': {'structuredMarkup': [{'mediaType': 'application/ld+json',
                                                 'uri': 'http://skos.um.es/unescothes/C01194/jsonld'},
                                                {'mediaType': 'text/turtle',
                                                 'uri': 'http://skos.um.es/unescothes/C01194/turtle'}],
                           'updateFrequency': 'yearly'},
 'base': {'author': 'Ocean Protocol Foundation Ltd.',
          'categories': ['white-papers'],
          'compression': '',
          'copyrightHolder': 'Ocean Protocol Foundation Ltd.',
          'dateCreated': '2012-10-10T17:00:000Z',
          'description': 'Introduce the main concepts and vision behind ocean '
                         'protocol',
          'files': [{'checksum': 'efb2c764274b745f5fc37f97c6b0e761',
                     'checksumType': 'MD5',
                     'contentLength': '4535431',
               

Note that the price is included in the Metadata! This will be purchase price you are placing on the asset. You can
Alter the metadata object at any time before publishing.

In [8]:
print("Price of Asset:", metadata['base']['price'])
metadata['base']['price'] = 9
print("Updated price of Asset:", metadata['base']['price'])

Price of Asset: 23
Updated price of Asset: 9


Let's inspect another important component of your metadata - the actual asset files. The files of an asset are
described by valid URL's. You are responsible for ensuring the URL's are alive. Files may have additional
information, including a checksum, length, content type, etc.

In [9]:
for i, file in enumerate(metadata['base']['files']):
    print("Asset link {}: {}".format( i, file['url']))

Asset link 0: https://testocnfiles.blob.core.windows.net/testfiles/testzkp.pdf
Asset link 1: s3://ocean-test-osmosis-data-plugin-dataseeding-1537375953/data.txt
Asset link 2: http://www3.cs.stonybrook.edu/~algorith/implement/graphbase/distrib/cweb3.4e.tar.gz


## Section 3 Publish the asset
With this metadata object prepared, we are ready to publish the asset into Ocean Protocol.

The result will be an ID string (DID) registered into the smart contract, and a DID Document stored in Aquarius.
The asset URLS's are encrypted upon publishing.

In [10]:
ddo = ocn.assets.create(metadata, publisher_acct)
registered_did = ddo.did
print("New asset registered at", registered_did)

20 - ocean_assets    - create          - Content urls encrypted successfully 0x2f583676432b67366bc854071918bb4c9e4de22c83708a87cc27cd73366cdce62d544f429b96b8a8a06796cf174c5e1dec734e61497992e0752336d80088705429653a5bbd67f69e863e4aba14df24525315694a9719ed84f9d0f87219053eddceb6d521a7abdb91eac04f5e22852ef6fab6d3eba51e0ef4242d600c116b13dc128853c9cd6b1e65430057d764f8f0aee9cedff3e55843f39f9f7b29af82be73a79e096d1e8a4a9e81010541d3f6d5ea6fe03eddd4e9a94c919dee3195ffdbd97b132c07c1fbe00377cfda106521a32ea1627a56e66d8813c323c12c44c785e51413c006f955de8d6e1d4799a89825dd19f273cf04ba1ad97915ec069e117aea6e24e597f68efc0e7a433062b265ef389b945f575793210fc5512a22c87b8e935be6130e4f1351ba318bda1f3672d6eaaf91c2d62da0e225841876fc155abae50f99bf9f92ded9cac29d78cc65db099dac2f2d79b93d97bd2beb011bac5004476c3f2bd8e89429a08cf4bffce4e26e40bc75ea36ff9d52c76f05481ac9b5254307d1ef3e598ac912196d406453d952d20a2e5ea84afb08e307bc4aa254b07defa229c4a7b50c4c554095acdfddeb03eb003494753c731e90807a748a6692f641404b7142291c62df28ed7c481

New asset registered at did:op:4178ae2d31704c52bfbb204c62704252dec1b0384e69444aaf79abc3b023e569


Inspect the new DDO. We can retrieve the DDO as a dictionary object, feel free to explore the DDO in the cell below!

In [11]:
ddo_dict = ddo.as_dictionary()
print("DID:", ddo.did)
print("Services within this DDO:")
for svc in ddo_dict['service']:
    print(svc['type'], svc['serviceEndpoint'])

DID: did:op:4178ae2d31704c52bfbb204c62704252dec1b0384e69444aaf79abc3b023e569
Services within this DDO:
Authorization https://secret-store.dev-ocean.com
Access https://nginx-brizo.dev-ocean.com//api/v1/brizo/services/consume
Metadata https://nginx-aquarius.dev-ocean.com//api/v1/aquarius/assets/ddo/did:op:4178ae2d31704c52bfbb204c62704252dec1b0384e69444aaf79abc3b023e569


Note that the 'files' attribute has been modified - all URL's are now removed, and a new 'encryptedFiles'
attribute is created to store the actual URLs.

In [12]:
for file_attrib in ddo.metadata['base']['files']:
    assert 'url' not in file_attrib
print("Encrypted files decrypt on purchase! Cipher text: [{}...] . ".format(ddo.metadata['base']['encryptedFiles'][:50]))

Encrypted files decrypt on purchase! Cipher text: [0x2f583676432b67366bc854071918bb4c9e4de22c83708a87...] . 


## Section 4: Verify your asset
Now, let's verify that this asset exists in the MetaData storage.

A call to assets.resolve() will call the Aquarius service and retrieve the DID Document

In [13]:
#+attr_jupyter: some cell metadata stuff
#+attr_jupyter: some other metadata stuff

#TODO: Better handling based on reciept
print("Wait for the transaction to complete!")
sleep(10)

Wait for the transaction to complete!


In [14]:
ddo = ocn.assets.resolve(registered_did)
print("Asset '{}' resolved from Aquarius metadata storage: {}".format(ddo.did,ddo.metadata['base']['name']))

Asset 'did:op:4178ae2d31704c52bfbb204c62704252dec1b0384e69444aaf79abc3b023e569' resolved from Aquarius metadata storage: Ocean protocol white paper


Similarly, we can verify that this asset is registered into the blockchain, and that you are the owner.

In [15]:
# We need the pure ID string as in the DID registry (a DID without the prefixes)
asset_id = squid_py.did.did_to_id(registered_did)
owner = ocn._keeper.did_registry.contract_concise.getDIDOwner(asset_id)
print("Asset ID", asset_id, "owned by", owner)
assert str.lower(owner) == str.lower(publisher_acct.address)

Asset ID 4178ae2d31704c52bfbb204c62704252dec1b0384e69444aaf79abc3b023e569 owned by 0x442aCf84E05459e13595f372C49D4a15E95eA4E7


Congratulations on publishing an Asset into Ocean Protocol!

Next, let's search for our assets in Ocean Protocol